<a href="https://colab.research.google.com/github/IAMPathak2702/Loan_status_prediction-TFX-pipeline/blob/main/Loan_data_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

In [ ]:
!pip install tfx

In [ ]:
!pip uninstall shapely -y

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
print("TFX verison ==",tfx.__version__)
print("Tensorflow Version ==",tf.__version__)

1.14.0
2.13.1


In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: loan_data.csv           


In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "loan_data.csv")

In [ ]:
!head {_data_filepath}

Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
LP001024,Male,Yes,2,Graduate,No,3200,700.0,70.0,360.0,1.0,Urban,Y
LP001027,Male,Yes,2,Graduate,,2500,1840.0,109.0,360.0,1.0,Urban,Y
LP001029,Male,No,0,Graduate,No,1853,2840.0,114.0,360.0,1.0,Rural,N
LP001030,Male,Yes,2,Graduate,No,1299,1086.0,17.0,120.0,1.0,Urban,Y


In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-data3_cb9ycr/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/CsvExampleGen/examples/1


In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "ApplicantIncome"
    value {
      int64_list {
        value: 4583
      }
    }
  }
  feature {
    key: "CoapplicantIncome"
    value {
      float_list {
        value: 1508.0
      }
    }
  }
  feature {
    key: "Credit_History"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "Dependents"
    value {
      bytes_list {
        value: "1"
      }
    }
  }
  feature {
    key: "Education"
    value {
      bytes_list {
        value: "Graduate"
      }
    }
  }
  feature {
    key: "Gender"
    value {
      bytes_list {
        value: "Male"
      }
    }
  }
  feature {
    key: "LoanAmount"
    value {
      float_list {
        value: 128.0
      }
    }
  }
  feature {
    key: "Loan_Amount_Term"
    value {
      float_list {
        value: 360.0
      }
    }
  }
  feature {
    key: "Loan_ID"
    value {
      bytes_list {
        value: "LP001003"
      }
    }
  }
  feature {
    key: "Loan_Statu

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'ApplicantIncome',INT,required,single,-
'CoapplicantIncome',FLOAT,required,single,-
'Credit_History',FLOAT,required,,-
'Dependents',STRING,required,,'Dependents'
'Education',STRING,required,single,'Education'
'Gender',STRING,required,,'Gender'
'LoanAmount',FLOAT,required,single,-
'Loan_Amount_Term',FLOAT,required,,-
'Loan_ID',BYTES,required,single,-


,Values
Domain,
'Dependents',"'0', '1', '2', '3+'"
'Education',"'Graduate', 'Not Graduate'"
'Gender',"'Female', 'Male'"
'Loan_Status',"'N', 'Y'"
'Married',"'No', 'Yes'"
'Property_Area',"'Rural', 'Semiurban', 'Urban'"
'Self_Employed',"'No', 'Yes'"


In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2024-02-17T13_27_26.060488-gd8myq_i/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
context.show(example_validator.outputs['anomalies'])

In [ ]:
!cd /content/
!mkdir /content/tfx/
!mkdir /content/tfx/logs
!mkdir /content/tfx/metadata
!mkdir /content/tfx/serving_model
!mkdir /content/tfx/pipelines
!mkdir /content/tfx/data

In [ ]:
!ls /content/


archive.zip  loan_data.csv  sample_data


In [ ]:
import os

_tfx_root = os.path.join(os.getcwd(),"tfx")
_pipeline_root = os.path.join(_tfx_root,"piplelines")
_medadata_root = os.path.join(_tfx_root,"metadata.db")
_log_root = os.path.join(_tfx_root,"logs")
_model_root = os.path.join(_tfx_root,"models")
_data_root = os.path.join(_tfx_root,"data")
_serving_model_dir = os.path.join(_tfx_root,"serving_model")
_data_filepath = os.path.join(_data_root,"data_trans.csv")

_input_fn_module_file = "inputfn_trainer.py"
_constant_module_file = "constants_trainer.py"
_model_trainer_module_file = "model_trainer.py"

In [ ]:
!head {_data_filepath}

﻿"url","status","id","listPrice","baths","baths_full","baths_full_calc","beds","sqft","stories","sub_type","text","type","year_built"
"https://www.realtor.com/realestateandhomes-detail/10410-Daw-Collins-Rd_Cleveland_TX_77328_M97739-41616","for_sale","9773941616","240000","2","2","2","3","1190","1",,"Welcome home to your peaceful retreat nestled on 2 acres of beautiful land! This charming home offers the perfect blend of comfort and serenity. Step inside to discover a cozy living space where relaxation meets functionality. The open layout seamlessly connects the living room, dining area, and kitchen, creating an inviting atmosphere for everyday living and entertaining. Outside, you'll find a spacious yard with plenty of room to roam and explore. Plus, the property includes a handy storage shed with electricity, and insulation perfect for storing tools, equipment, or creating your own workshop.Located in a peaceful setting, yet just a short drive from amenities and attractions, this prop

In [ ]:
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from typing import Dict, List, Text

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [ ]:
context = InteractiveContext(pipeline_name="tfx_pipline",
                             pipeline_root=_tfx_root,
                             )

In [ ]:
example_gen = CsvExampleGen(
    input_base=_data_)

In [ ]:
context.run(example_gen)

RuntimeError: Files in same split /content/tfx/data/* have different header.

In [ ]:
import urllib.request
import tempfile
import os

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/tfx-datahefg34s0/data.csv', <http.client.HTTPMessage at 0x780c1bf63eb0>)

In [ ]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Create an InteractiveContext instance
context = InteractiveContext()

# Define the input base directory containing your data
input_base = '/tmp/tfx-datahefg34s0/data.csv'

# Define the split pattern
split_pattern_train = 'train/*'
split_pattern_eval = 'eval/*'

# Print the input base directory and split patterns for debugging
print("Input base directory:", input_base)
print("Split pattern for training data:", split_pattern_train)
print("Split pattern for evaluation data:", split_pattern_eval)




Input base directory: /tmp/tfx-datahefg34s0/data.csv
Split pattern for training data: train/*
Split pattern for evaluation data: eval/*


RuntimeError: Split pattern /tmp/tfx-datahefg34s0/data.csv/* does not match any files.

In [ ]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
from tfx.orchestration import pipeline
from tfx.proto import example_gen_pb2

# Define your record_defaults

# Define your paths
data_path = '/content/real_estate_texas_500_2024.csv'
output_dir = '/content/output'

# Define CsvExampleGen component with record_defaults
example_gen = CsvExampleGen(input_base=data_path)

# Define the rest of your pipeline components
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])

context.run(example_gen)
context.run(statistics_gen)
context.run(schema_gen)
context.run(example_validator)


RuntimeError: Split pattern /content/real_estate_texas_500_2024.csv/* does not match any files.

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
# Set up an InteractiveContext, this will allow you to run the pipeline interactively.
context = InteractiveContext()

In [ ]:
!pip install tfx.utils.dsl_utils

ERROR: Could not find a version that satisfies the requirement tfx.utils.dsl_utils (from versions: none)
ERROR: No matching distribution found for tfx.utils.dsl_utils


In [ ]:
from tfx.components import CsvExampleGen
# from tfx.utils.dsl_utils import external_input/

# Assuming data_dir is the base directory containing your CSV files
data_dir = '/content/loan_data.csv'

# Create a CsvExampleGen component
example_gen = CsvExampleGen(input_base=data_dir)


In [ ]:
print(example_gen.spec.input_base)

AttributeError: 'FileBasedExampleGenSpec' object has no attribute 'input_base'

In [ ]:
context.run(example_gen)

RuntimeError: Split pattern /content/loan_data.csv/* does not match any files.

In [ ]:
!ls -l /content/data.csv



-rw-r--r-- 1 root root 22573 Feb 12 10:23 /content/data.csv


In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, ResolverNode, Evaluator, Pusher
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2

# Set up an InteractiveContext, this will allow you to run the pipeline interactively.
context = InteractiveContext()

# Path to your CSV dataset containing missing values
csv_path = "path/to/your/loan_approval_dataset.csv"

# Create ExampleGen component to ingest data
example_gen = CsvExampleGen(input_base=csv_path)

# Compute statistics over data for visualization and schema generation
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

# Infer schema from statistics
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)

# Validate data based on schema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Preprocess and transform the data
# You can define your transformation logic here to handle missing values, scaling, etc.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'])

# Train the model
trainer = Trainer(
    module_file='path/to/your/trainer_module.py',  # Path to your trainer module
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=5))

# Resolve artifacts produced by Trainer to use for model validation
resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=tfx.dsl.experimental.latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing))

# Evaluate the trained model
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    # Change it to use the 'transformed_examples' for better evaluation.
    # examples=transform.outputs['transformed_examples'],
    # Change it to use the 'transformed_examples' for better evaluation.
    eval_config=tfx.utils.dsl_utils.marshal_config(eval_config),
)

# Push the validated model to a deployment target
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(base_directory='your/desired/output/path')))

# Run the TFX pipeline
context.run(example_gen)
context.run(statistics_gen)
context.run(schema_gen)
context.run(example_validator)
context.run(transform)
context.run(trainer)
context.run(resolver)
context.run(evaluator)
context.run(pusher)


In [ ]:
import math
import os
import pprint

import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print('TF: {}'.format(tf.__version__))

import apache_beam as beam
print('Beam: {}'.format(beam.__version__))

import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
print('Transform: {}'.format(tft.__version__))

from tfx_bsl.public import tfxio
from tfx_bsl.coders.example_coder import RecordBatchToExamplesEncoder

TF: 2.13.1
Beam: 2.50.0
Transform: 1.14.0


In [ ]:
CATEGORICAL_FEATURE_KEYS=[
    'Gender',
    'Married',
    'Dependents',
    'Education',
    'Self_Employed',
    'Property_Area',
    'Loan_Status']
NUMERIC_FEATURE_KEYS=[
    'CoapplicantIncome',
    'LoanAmount',
    'Loan_Amount_Term',
    'Credit_History']

ORDERED_COLUMNS_CSV = [
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Property_Area',
 ]

LABEL_KEY = 'Loan_Status'

In [ ]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [ ]:
dict_data = dict(data.loc[0])
dict_data

{'Gender': 'Male',
 'Married': 'Yes',
 'Dependents': '1',
 'Education': 'Graduate',
 'Self_Employed': 'No',
 'ApplicantIncome': 4583,
 'CoapplicantIncome': 1508.0,
 'LoanAmount': 128.0,
 'Loan_Amount_Term': 360.0,
 'Credit_History': 1.0,
 'Property_Area': 'Rural',
 'Loan_Status': 'N'}

In [ ]:
COLUMN_DEFAULTS = [
  '' if isinstance(v, str) else 0.0
  for v in  dict(data.loc[1]).values()]


COLUMN_DEFAULTS

['', '', '', '', '', 0.0, 0.0, 0.0, 0.0, 0.0, '', '']

In [ ]:
RAW_DATA_FEATURE_SPEC = dict(
    [(name, tf.io.FixedLenFeature([], tf.string))
     for name in CATEGORICAL_FEATURE_KEYS] +
    [(name, tf.io.FixedLenFeature([], tf.float32))
     for name in NUMERIC_FEATURE_KEYS] +
    [(LABEL_KEY, tf.io.FixedLenFeature([], tf.string))]
)

SCHEMA = tft.DatasetMetadata.from_feature_spec(RAW_DATA_FEATURE_SPEC).schema
SCHEMA

feature {
  name: "CoapplicantIncome"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Credit_History"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Dependents"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Education"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Gender"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "LoanAmount"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Loan_Amount_Term"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Loan_Status"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Married"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "Property_Area"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  s